# Load the given JSON into pyspark data-frame

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark= SparkSession.builder.appName("Q2JSON").getOrCreate()


25/04/22 12:45:23 WARN Utils: Your hostname, prabina-bhushal resolves to a loopback address: 127.0.1.1; using 192.168.1.191 instead (on interface wlx54ef33f6c819)
25/04/22 12:45:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/22 12:45:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [19]:
df_pyspark = spark.read.option('header','option').option('inferSchema','true').json('MOCK_DATA.json')
df_pyspark.show()

+--------------------+-------------+-----------------+-------------+--------------------+---------------+---------------+---------------+-----------------------+------------+
|     _corrupt_record|billing_class|billing_code_type|billing_codee|         description|expiration_date|negotiated_rate|negotiated_type|negotiation_arrangement|service_code|
+--------------------+-------------+-----------------+-------------+--------------------+---------------+---------------+---------------+-----------------------+------------+
|[{"billing_class"...|         NULL|             NULL|         NULL|                NULL|           NULL|           NULL|           NULL|                   NULL|        NULL|
|                NULL|          NVA|             NULL|   54868-6381|Occup of pk-up/va...|     2023/10/27|          $1.89|             CO|             49288-0290|          ID|
|                NULL|          JEQ|             SNJK|    55315-329|Posterior subluxa...|     2023/03/22|          $6.07|    

In [20]:
print(type(df_pyspark))

<class 'pyspark.sql.dataframe.DataFrame'>


# Remove name, description and expiration date

In [21]:
df_pyspark = df_pyspark.drop('name','description','expiration_date')
df_pyspark.show()

+--------------------+-------------+-----------------+-------------+---------------+---------------+-----------------------+------------+
|     _corrupt_record|billing_class|billing_code_type|billing_codee|negotiated_rate|negotiated_type|negotiation_arrangement|service_code|
+--------------------+-------------+-----------------+-------------+---------------+---------------+-----------------------+------------+
|[{"billing_class"...|         NULL|             NULL|         NULL|           NULL|           NULL|                   NULL|        NULL|
|                NULL|          NVA|             NULL|   54868-6381|          $1.89|             CO|             49288-0290|          ID|
|                NULL|          JEQ|             SNJK|    55315-329|          $6.07|             BR|              10544-105|          CN|
|                NULL|          SSO|             SNLO|    13537-015|          $4.78|             BR|              53489-536|          CN|
|                NULL|          CK

# Remove hyphen from billing code, negotiation arrangements and $ from negotiated rate

In [22]:
from pyspark.sql.functions import expr,hash,lit

df_cleaned = df_pyspark.withColumn("billing_codee",expr("REPLACE(billing_codee,'-','')"))\
                .withColumn("negotiation_arrangement",expr("REPLACE(negotiation_arrangement,'-','')"))

df_cleaned.show()

+--------------------+-------------+-----------------+-------------+---------------+---------------+-----------------------+------------+
|     _corrupt_record|billing_class|billing_code_type|billing_codee|negotiated_rate|negotiated_type|negotiation_arrangement|service_code|
+--------------------+-------------+-----------------+-------------+---------------+---------------+-----------------------+------------+
|[{"billing_class"...|         NULL|             NULL|         NULL|           NULL|           NULL|                   NULL|        NULL|
|                NULL|          NVA|             NULL|    548686381|          $1.89|             CO|              492880290|          ID|
|                NULL|          JEQ|             SNJK|     55315329|          $6.07|             BR|               10544105|          CN|
|                NULL|          SSO|             SNLO|     13537015|          $4.78|             BR|               53489536|          CN|
|                NULL|          CK

# Remove $ from negotiated rate

In [25]:
df_replaced=df_cleaned.withColumn("negotiated_rate",expr("REPLACE(negotiated_rate,'$','')"))
df_replaced.show()

+--------------------+-------------+-----------------+-------------+---------------+---------------+-----------------------+------------+
|     _corrupt_record|billing_class|billing_code_type|billing_codee|negotiated_rate|negotiated_type|negotiation_arrangement|service_code|
+--------------------+-------------+-----------------+-------------+---------------+---------------+-----------------------+------------+
|[{"billing_class"...|         NULL|             NULL|         NULL|           NULL|           NULL|                   NULL|        NULL|
|                NULL|          NVA|             NULL|    548686381|           1.89|             CO|              492880290|          ID|
|                NULL|          JEQ|             SNJK|     55315329|           6.07|             BR|               10544105|          CN|
|                NULL|          SSO|             SNLO|     13537015|           4.78|             BR|               53489536|          CN|
|                NULL|          CK

# Use hashing for service code column

In [26]:
df_hash=df_replaced.withColumn('service_code',hash('service_code'))
df_hash.show()

+--------------------+-------------+-----------------+-------------+---------------+---------------+-----------------------+------------+
|     _corrupt_record|billing_class|billing_code_type|billing_codee|negotiated_rate|negotiated_type|negotiation_arrangement|service_code|
+--------------------+-------------+-----------------+-------------+---------------+---------------+-----------------------+------------+
|[{"billing_class"...|         NULL|             NULL|         NULL|           NULL|           NULL|                   NULL|          42|
|                NULL|          NVA|             NULL|    548686381|           1.89|             CO|              492880290|  -214147340|
|                NULL|          JEQ|             SNJK|     55315329|           6.07|             BR|               10544105|   446854828|
|                NULL|          SSO|             SNLO|     13537015|           4.78|             BR|               53489536|   446854828|
|                NULL|          CK

# Remove the row if 'billing_code' is null

In [27]:
df_result=df_hash.filter(df_hash.billing_codee.isNotNull())
df_result.show()

+---------------+-------------+-----------------+-------------+---------------+---------------+-----------------------+------------+
|_corrupt_record|billing_class|billing_code_type|billing_codee|negotiated_rate|negotiated_type|negotiation_arrangement|service_code|
+---------------+-------------+-----------------+-------------+---------------+---------------+-----------------------+------------+
|           NULL|          NVA|             NULL|    548686381|           1.89|             CO|              492880290|  -214147340|
|           NULL|          JEQ|             SNJK|     55315329|           6.07|             BR|               10544105|   446854828|
|           NULL|          SSO|             SNLO|     13537015|           4.78|             BR|               53489536|   446854828|
|           NULL|          CKR|             NULL|     42508138|           4.35|             US|               59572425|   519803259|
|           NULL|          CXH|             CYHC|     52533120|      

# Replace all the null values in billing_class to ‘I’

In [28]:
df_filter=df_result.fillna({'billing_class':'I'})
df_filter.show()

+---------------+-------------+-----------------+-------------+---------------+---------------+-----------------------+------------+
|_corrupt_record|billing_class|billing_code_type|billing_codee|negotiated_rate|negotiated_type|negotiation_arrangement|service_code|
+---------------+-------------+-----------------+-------------+---------------+---------------+-----------------------+------------+
|           NULL|          NVA|             NULL|    548686381|           1.89|             CO|              492880290|  -214147340|
|           NULL|          JEQ|             SNJK|     55315329|           6.07|             BR|               10544105|   446854828|
|           NULL|          SSO|             SNLO|     13537015|           4.78|             BR|               53489536|   446854828|
|           NULL|          CKR|             NULL|     42508138|           4.35|             US|               59572425|   519803259|
|           NULL|          CXH|             CYHC|     52533120|      

# Rename all the column as given. billing_class>>bCls  , billing_code>>bC, billing_code_type>> bCT , negotiated_rate>> negR, negotiated_type>> negT, negotiation_arrangements>> negA, service_code>> poSH

In [29]:
df_renamed=df_filter.withColumnRenamed('billing_class','bcIs')\
            .withColumnRenamed('billing_codee','bC')\
            .withColumnRenamed('billing_code_type','bCT')\
            .withColumnRenamed('negotiated_rate','negR')\
            .withColumnRenamed('negotiated_type','negT')\
            .withColumnRenamed('negotiation_arrangement','negA')\
            .withColumnRenamed('service_code','poSH')
df_renamed.show()

+---------------+----+----+---------+----+----+---------+-----------+
|_corrupt_record|bcIs| bCT|       bC|negR|negT|     negA|       poSH|
+---------------+----+----+---------+----+----+---------+-----------+
|           NULL| NVA|NULL|548686381|1.89|  CO|492880290| -214147340|
|           NULL| JEQ|SNJK| 55315329|6.07|  BR| 10544105|  446854828|
|           NULL| SSO|SNLO| 13537015|4.78|  BR| 53489536|  446854828|
|           NULL| CKR|NULL| 42508138|4.35|  US| 59572425|  519803259|
|           NULL| CXH|CYHC| 52533120|1.35|  CA| 41250871| -214147340|
|           NULL| KTT|EFKT| 36800952|7.52|  FI| 59886410|  446854828|
|           NULL|   I|NULL| 11344999|2.94|  JP| 76509151|  446854828|
|           NULL| DAL|NULL|369871207|4.07|NULL| 11410020| 1062621904|
|           NULL| GDA|NULL| 76138106|9.05|  CF| 65954538| 1817792724|
|           NULL| AEX|KAEX| 53329809|3.61|  US| 50991216|  175481947|
|           NULL|   I|SPHI| 04960760|4.79|  PE| 49884641|-1524333386|
|           NULL| DU

# Add a whole new column named billing_code_modifier and rename it to 'mdH'

In [30]:
df_col=df_renamed.withColumn('billing_code_modifier',lit(1426636))
df_col=df_col.withColumnRenamed('billing_code_modifier','mdH')

# Use default value 1426636 for mdH and hash the mdH column.

In [33]:
df_final=df_col.withColumn('mdH',hash('mdH'))
df_final.show()

+---------------+----+----+---------+----+----+---------+-----------+---------+
|_corrupt_record|bcIs| bCT|       bC|negR|negT|     negA|       poSH|      mdH|
+---------------+----+----+---------+----+----+---------+-----------+---------+
|           NULL| NVA|NULL|548686381|1.89|  CO|492880290| -214147340|678617635|
|           NULL| JEQ|SNJK| 55315329|6.07|  BR| 10544105|  446854828|678617635|
|           NULL| SSO|SNLO| 13537015|4.78|  BR| 53489536|  446854828|678617635|
|           NULL| CKR|NULL| 42508138|4.35|  US| 59572425|  519803259|678617635|
|           NULL| CXH|CYHC| 52533120|1.35|  CA| 41250871| -214147340|678617635|
|           NULL| KTT|EFKT| 36800952|7.52|  FI| 59886410|  446854828|678617635|
|           NULL|   I|NULL| 11344999|2.94|  JP| 76509151|  446854828|678617635|
|           NULL| DAL|NULL|369871207|4.07|NULL| 11410020| 1062621904|678617635|
|           NULL| GDA|NULL| 76138106|9.05|  CF| 65954538| 1817792724|678617635|
|           NULL| AEX|KAEX| 53329809|3.6

# Write that dataframe into parquet using coalesce (1)

In [34]:
df_output=df_final.coalesce(1).write.parquet('Result')